In [1]:
import environment
import agent
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from time import sleep
from util import score_func
import random
import torch

random.seed(77)
train = True
render = False
epoch = 800 if train else 1
agent = agent.DQNAgent(state_size=54, action_size=23, train=train, state_dict=None)
# agent = agent.DQNAgent(state_size=139, action_size=23, train=True, state_dict="weight/save_1.pth")
avg_score = 0

if train:
    score_history = []
    p_history = []
    q_history = []
    s_history = []
    c_history = []

if render:
    render_p_history = [] # 시험용
    render_q_history = [] # 시험용
    render_s_history = [] # 시험용
    render_c_history = [] # 시험용
    render_score_history = []

for i in tqdm(range(epoch)):
    env = environment.FactoryEnv()
    s = 0
    
    if train == False:
        submission = pd.read_csv("data/sample_submission.csv")
        actions = []
        process_n = []        

    for step in range(2184):       
        r = 0
        s = 0
        state = env.get_state()
        mask = agent.get_mask(env)
        action = agent.get_action(state,mask)
        
        if train == False:
            if i == epoch-1:
                action_name = env.action_map[action]
                action_process = 0
                if 'PROCESS' in action_name:
                    mylist = action_name.split("_")
                    action_name = mylist[0]
                    action_process = float(mylist[1])                
                if 'CHANGE' in action_name:
                    action_name = "CHANGE_" + str(env.prev_change+1) + str(action-3)
                actions.append(action_name)
                process_n.append(action_process)
      
        if render:
            render_score_history.append(env.prev_score)
            render_p_history.append(score_func(env.p, 32550830) * 100)
            render_q_history.append(score_func(env.q, 32550830) * 100)
            render_s_history.append(score_func(env.s_t, 2184) / (1 + 0.1*env.s_n) * 100)
            render_c_history.append(score_func(env.c_t, 2184) / (1 + 0.1*env.c_n) * 100)
            
        next_state, reward, done = env.step(action)

        if train:
            agent.append_sample(state, action, reward, next_state, done)
            agent.train_model()
            s += env.prev_score

        if done:
            break
            
            
    if render:
        fig = plt.figure()
        plt.rcParams["figure.figsize"] = (25,6)
        score, = plt.plot(render_score_history, label = 'score')
        over, = plt.plot(render_p_history, label = 'over')
        under, = plt.plot(render_q_history, label = 'under')
        stop, = plt.plot(render_s_history, label = 'stop')
        change, = plt.plot(render_c_history, label = 'change')
        plt.legend([score, over, under, stop, change], ["score", "over", "under", "stop", "change"])
        plt.savefig(f"graph/save_render_{env.prev_score}.png")
        plt.clf()
        
#     if not train:
#         aaa = list()
#         bbb = list()
#         for j in range(4):
#             env.J[j].save_graph(f"performance/performance{j}.png")
#             info = np.array(env.J[j].info)
#             t, w, o = info[:, 0], info[:, 1], info[:, 2]
#             aaa.append(sum(w)/t[-1])
#             bbb.append(sum(o)/t[-1])
#         print(sum(aaa)/len(aaa))
#         print(sum(bbb)/len(bbb))
    
    if train:
        avg_score = s if avg_score == 0 else avg_score * 0.9 + s * 0.1
        score_history.append(env.prev_score)
        p_history.append(score_func(env.p, 32550830) * 100) 
        q_history.append(score_func(env.q, 32550830) * 100) 
        s_history.append(score_func(env.s_t, 2184) / (1 + 0.1*env.s_n) * 100) 
        c_history.append(score_func(env.c_t, 2184) / (1 + 0.1*env.c_n) * 100) 
        if i % 100 == 99:
            torch.save(agent.model.state_dict(), f"weight/save_{i + 1}_{env.prev_score}.pth")    
            
            # 그래프 그리기
            fig = plt.figure()
            plt.rcParams["figure.figsize"] = (25,6)
            score, = plt.plot(score_history, label = 'score')
            over, = plt.plot(p_history, label = 'over')
            under, = plt.plot(q_history, label = 'under')
            stop, = plt.plot(s_history, label = 'stop')
            change, = plt.plot(c_history, label = 'change')
            
            plt.legend([score, over, under, stop, change], ["score", "over", "under", "stop", "change"])
            
            plt.savefig(f"graph/save_{i + 1}_{env.prev_score}.png")
            plt.clf()
    

100%|██████████████████████████████████████████████████████████████████████████████| 800/800 [5:09:13<00:00, 23.19s/it]


In [2]:
if train == False:
    submission.loc[:, "Event_A"] = actions
    submission.loc[:, "MOL_A"] = process_n
    submission.loc[:, "Event_B"] = actions
    submission.loc[:, "MOL_B"] = process_n

    PRTs = submission[["PRT_1", "PRT_2", "PRT_3", "PRT_4"]].values
    for i in (np.array(env.PRT) - np.array([0,0,554])):
        if i[2] >= 0:
            PRTs[i[2], i[0]] = i[1]
    PRTs = np.ceil(PRTs * 1.1)
    PRTs = PRTs.astype(int)
    submission.loc[:, "PRT_1":"PRT_4"] = PRTs
    submission.to_csv(f"submission/save_{env.prev_score}.csv", index=False)    